In [368]:
import pandas as pd

# Load the raw dataset
df = pd.read_excel("Address_Sorted_File_2.xlsx")

In [369]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381 entries, 0 to 380
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   S.NO.               381 non-null    int64 
 1   NAME                381 non-null    object
 2   AGE                 381 non-null    object
 3   SEX                 381 non-null    object
 4   ADDRESS             381 non-null    object
 5   BLOOD GROUP         381 non-null    object
 6   DISEASE ASSOCIATED  381 non-null    object
dtypes: int64(1), object(6)
memory usage: 21.0+ KB


In [370]:
df.columns = df.columns.str.strip().str.upper()

In [371]:
df.head()

,S.NO.,NAME,AGE,SEX,ADDRESS,BLOOD GROUP,DISEASE ASSOCIATED
0,1,ALIVIA PIPLAI,30 YRS.,F,CHAKDAHA,O+,"FATTY LIVER, DIGESTIVE"
1,2,BULBUL PIPLAI,63 YRS,F,CHAKDAHA,O-,"OSTEOPORESIS,DYSENTRY"
2,3,MONOTOSH PIPLAI,74 YRS.,M,CHAKDAHA,O+,"ATRIAL FIBRATION,HIGH BP. CHLOESTROL"
3,4,ESHA PIPLAI,36 YRS.,F,CHAKDAHA,O-,NORMAL GASTRIC PROBLEM
4,5,ALO RANI DAS,89 YRS.,F,CHAKDAHA,A+,"DIABETIC, HIGH BP, ULCERS"


In [372]:
# Clean age column (optional but useful)
df["AGE"] = df["AGE"].astype(str).str.extract(r"(\d+)").astype(float)

In [373]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381 entries, 0 to 380
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   S.NO.               381 non-null    int64  
 1   NAME                381 non-null    object 
 2   AGE                 381 non-null    float64
 3   SEX                 381 non-null    object 
 4   ADDRESS             381 non-null    object 
 5   BLOOD GROUP         381 non-null    object 
 6   DISEASE ASSOCIATED  381 non-null    object 
dtypes: float64(1), int64(1), object(5)
memory usage: 21.0+ KB


In [374]:
df.rename(columns={'DISEASE ASSOCIATED': 'DISEASES'}, inplace=True)

In [375]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381 entries, 0 to 380
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   S.NO.        381 non-null    int64  
 1   NAME         381 non-null    object 
 2   AGE          381 non-null    float64
 3   SEX          381 non-null    object 
 4   ADDRESS      381 non-null    object 
 5   BLOOD GROUP  381 non-null    object 
 6   DISEASES     381 non-null    object 
dtypes: float64(1), int64(1), object(5)
memory usage: 21.0+ KB


In [376]:
persons_studied = df.groupby("ADDRESS")["S.NO."].nunique().reset_index(name="Persons_Studied")
persons_studied

,ADDRESS,Persons_Studied
0,CHAKDAHA,151
1,KALYANI,21
2,NAIHATI,18
3,PALPARA,150
4,SIMURALI,41


In [377]:
# Drop rows with missing address or diseases
df = df.dropna(subset=["ADDRESS", "DISEASES"])
df = df[df["DISEASES"].str.upper().str.strip() != "NIL"]

In [378]:
df.head()

,S.NO.,NAME,AGE,SEX,ADDRESS,BLOOD GROUP,DISEASES
0,1,ALIVIA PIPLAI,30.0,F,CHAKDAHA,O+,"FATTY LIVER, DIGESTIVE"
1,2,BULBUL PIPLAI,63.0,F,CHAKDAHA,O-,"OSTEOPORESIS,DYSENTRY"
2,3,MONOTOSH PIPLAI,74.0,M,CHAKDAHA,O+,"ATRIAL FIBRATION,HIGH BP. CHLOESTROL"
3,4,ESHA PIPLAI,36.0,F,CHAKDAHA,O-,NORMAL GASTRIC PROBLEM
4,5,ALO RANI DAS,89.0,F,CHAKDAHA,A+,"DIABETIC, HIGH BP, ULCERS"


In [379]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 268 entries, 0 to 379
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   S.NO.        268 non-null    int64  
 1   NAME         268 non-null    object 
 2   AGE          268 non-null    float64
 3   SEX          268 non-null    object 
 4   ADDRESS      268 non-null    object 
 5   BLOOD GROUP  268 non-null    object 
 6   DISEASES     268 non-null    object 
dtypes: float64(1), int64(1), object(5)
memory usage: 16.8+ KB


In [380]:
df["DISEASES"] = df["DISEASES"].str.upper().str.strip()

In [381]:
# Save person count before exploding



In [382]:
df = df.assign(DISEASES=df["DISEASES"].str.split(r",\s*")).explode("DISEASES")

In [383]:
df.head()

,S.NO.,NAME,AGE,SEX,ADDRESS,BLOOD GROUP,DISEASES
0,1,ALIVIA PIPLAI,30.0,F,CHAKDAHA,O+,FATTY LIVER
0,1,ALIVIA PIPLAI,30.0,F,CHAKDAHA,O+,DIGESTIVE
1,2,BULBUL PIPLAI,63.0,F,CHAKDAHA,O-,OSTEOPORESIS
1,2,BULBUL PIPLAI,63.0,F,CHAKDAHA,O-,DYSENTRY
2,3,MONOTOSH PIPLAI,74.0,M,CHAKDAHA,O+,ATRIAL FIBRATION


In [384]:
disease_mapping = {
    # ✅ Diabetes Variants
    "DIABETIC": "DIABETES",
    "DIABTIC": "DIABETES",
    "DIABETIC MELLITUS": "DIABETES",
    "DIABETICS": "DIABETES",
    "DIABETIS": "DIABETES",

    #DIGESTIVE
    "DIGESTIVE":"BELLY UPSET",

    # ✅ Cholesterol
    "CHOLESTEROL": "HIGH CHOLESTEROL",
    "CHLOESTROL": "HIGH CHOLESTEROL",
    "HIGH BP. CHLOESTROL": "HIGH CHOLESTEROL",
    "HIGH CHOLOESTROL": "HIGH CHOLESTEROL",
    "HIGH CHOLESTROL":"HIGH CHOLESTEROL",
    "CHOLOESTROL":"HIGH CHOLESTEROL",

     "ASTHAMA":"ASTHMA",
     "HAD OVARIAN CYST OPERATION":"OVARIAN CYST",

    # ✅ Allergy
    "ALLERGIC": "ALLERGY",
    "ALLERGIES": "ALLERGY",
    "ALLERGY PROBLEM": "ALLERGY",
    "BLOOD ALLERGY": "ALLERGY",

    # ✅ Arthritis
    "ARTHRITIS": "ARTHIRITIS",
    "ARITHRITIS":"ARTHIRITIS",

    # ✅ Ulcer / Gastric
    "PEPTIC ULCER": "ULCER",
    "GASTRIC PROBLEM": "ULCER",
    "NORMAL GASTRIC PROBLEM": "ULCER",
    "ULCERS": "ULCER",

    # ✅ Thyroid
    "THYROID": "THYROID DISORDER",
    "HYPOTHYROIDISM": "THYROID DISORDER",

    # ✅ Hypertension / BP
    "HIGH BP": "HYPERTENSION",
    "BP": "HYPERTENSION",
    "HIGH BLOOD PRESSURE": "HYPERTENSION",
    "HYPERTENSIVE": "HYPERTENSION",
    "BLOOD PRESSURE": "HYPERTENSION",
    "HIGH BLOOP PRESSURE": "HYPERTENSION",

    "ANAEMIC":"BLOOD DISORDER",

    # ✅ Cough/Cold
    "CHRONIC COUGH": "COLD & COUGH",
    "ACUTE COUGH": "COLD & COUGH",
    "COLD": "COLD & COUGH",
    "ACUTE COUGH AND COLD" :"COUGH & COLD",
    "CHRONIC COUGH AND COLD":"COUGH & COLD",
    "COUGH AND COLD":"COUGH & COLD",

    # ✅ Liver
    "CIRRHOSIS OF LIVER": "LIVER DISEASE",
    "CIRROHOSIS OF LIVER": "LIVER DISEASE",
    "LIVER": "LIVER DISEASE",
    "LIVER PROBLEM": "LIVER DISEASE",
    "HAD LIVER PROBLEM": "LIVER DISEASE",
    "FIBROSIS OF LIVER": "LIVER DISEASE",
    "HAVE LIVER PROBLEM": "LIVER DISEASE",
    "FATTY LIVER":"LIVER DISEASE",
    "FATTY LIVER" :"LIVER DISEASE",
    "KIDNEY AND LIVER PROBLEM":"LIVER DISEASE",
    "DETECTED WITH KIDNEY STONE AND FATTY LIVER":"LIVER DISEASE",
    "LIVER AND KIDNEY DISEASE":"LIVER DISEASE",

    # ✅ Kidney
    "KIDNEY": "KIDNEY DISEASE",
    "KIDNEY PROBLEM": "KIDNEY DISEASE",
    "HAD KIDNEY PROBLEM": "KIDNEY DISEASE",
    "HAD KIDNEY STONE OPERATION": "KIDNEY DISEASE",
    "DETECTED WITH KIDNEY STONE": "KIDNEY DISEASE",
    "KIDNEY CYST": "KIDNEY DISEASE",
    "KIDNEY FAILURE": "KIDNEY DISEASE",
    "KIDNEY MALFUNCTION": "KIDNEY DISEASE",
    "KIDNEY STONE OPERATION": "KIDNEY DISEASE",
    "KIDNEY STONE OPERATION.": "KIDNEY DISEASE",
    "KIDNEY STONE OPERATED": "KIDNEY DISEASE",
    "HAVE POLYCYSTIC KIDNEY DISEASE": "KIDNEY DISEASE",
    "ACUTE KIDNEY DISEASE ALONGWITH NUMEROUS KIDNEY STONE": "KIDNEY DISEASE",
    "KIDNEY AND LIVER PROBLEM":"KIDNEY DISEASE",
    "DETECTED WITH KIDNEY STONE AND FATTY LIVER":"KIDNEY DISEASE",
    "LIVER AND KIDNEY DISEASE":"KIDNEY DISEASE",

    #LUNGS
    "RESPIRATORY TRACT INFECTION":"LUNG DISEASE",
    "LUNGS PROBLEM":"LUNG DISEASE",
    # ✅ Heart
    "HAD HEART ATTACK": "HEART DISEASE",
    "HAD OPEN HEART SURGERY": "HEART DISEASE",
    "HEART PROBLEM": "HEART DISEASE",
    "HEART SURGERY": "HEART DISEASE",
    "OPEN HEART SURGERY": "HEART DISEASE",
    "OPEN HEART SYRGERY": "HEART DISEASE",
    "CARDIOVASCULAR DISEASE": "HEART DISEASE",

    # ✅ Gall bladder
    "HAD GALL BLADDER": "GALL BLADDER SURGERY",
    "HAD GALL BLADDER STONE OPERATION": "GALL BLADDER SURGERY",
    "GALL BLADDER STONE OPERATION": "GALL BLADDER SURGERY",
    "HAD GALL BLADDDER OPERATION": "GALL BLADDER SURGERY",
    "GALL BLADDER OPERATION": "GALL BLADDER SURGERY",

    # ✅ Cataract
    "CATARACT OPERATION": "CATARACT",
    "HAD CATARACT OPERATION": "CATARACT",

    # ✅ Spinal/Back
    "SPINAL CORD PROBLEM": "SPINAL ISSUE",
    "CYST IN SPINAL CORD": "SPINAL ISSUE",
    "BACK PAIN": "SPINAL ISSUE",
    "INJURY IN THE LOWER BACK BONE": "SPINAL ISSUE",
    "SPONDYLYSIS": "SPINAL ISSUE",

    # ✅ Nervous
    "NERVOUS BREAKDOWN": "NERVOUS ISSUE",
    "SUDDEN NERVOUS BREAKDOWN": "NERVOUS ISSUE",

    # ✅ PCOS / Hormonal
    "POLYCISTONIC OVARY": "PCOS",
    "POLYCYSTIC OVARIAN DISEASE": "PCOS",
    "POLYCYSTONIC OVARIAN DISEASE": "PCOS",
    "POLYCYSTONIC OVARY": "PCOS",
    "POLYCYSTONIC OVARY SYNDROME": "PCOS",
    "PCOD": "PCOS",
    "PCOS": "PCOS",

    # ✅ Misc Cleanup
    "HAD PAST HISTORY OF BREAST CANCER": "BREAST CANCER",
    "FREQUENT CHEST PAIN": "PAIN",
    "UNUSUAL CHEST PAIN": "PAIN",

    "FREQUENT BODY PAIN": "PAIN",
    "FREQUENT JOINT PAIN": "PAIN",

    "CHRONIC OBSTRUCTIVE PULMONARY DISORDER":"COPD",
    "BREATHING PROBLEM":"COPD",

    "LOW HEAMOGLOBIN": "IRON DEFICIENCY",
    "LOW IRON": "IRON DEFICIENCY",
    "LOW  IRON LEVEL": "IRON DEFICIENCY",

    "LOW PLATELET COUNT": "BLOOD DISORDER",
    "DEFICIENCY OF SODIUM": "ELECTROLYTE IMBALANCE",
    "POTASSIUM": "ELECTROLYTE IMBALANCE",
    "HIGH UREA IN BLOOD": "KIDNEY DISEASE",

    "HANDIKAPPED": "DISABILITY",
    "PARTIALLY BLIND": "VISION DISORDER",
    "EYE SIGHT PROBLEM": "VISION DISORDER",

    "NO CHECKUP": "NIL",
    "NIL": "NIL"
}



In [385]:
df["DISEASE"] = df["DISEASES"].replace(disease_mapping)

In [386]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 425 entries, 0 to 379
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   S.NO.        425 non-null    int64  
 1   NAME         425 non-null    object 
 2   AGE          425 non-null    float64
 3   SEX          425 non-null    object 
 4   ADDRESS      425 non-null    object 
 5   BLOOD GROUP  425 non-null    object 
 6   DISEASES     425 non-null    object 
 7   DISEASE      425 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 29.9+ KB


In [387]:
# Count affected per (ADDRESS, DISEASE)
affected = df.groupby(["DISEASE", "ADDRESS"]).size().reset_index(name="Affected_Persons")

In [388]:
affected

,DISEASE,ADDRESS,Affected_Persons
0,,CHAKDAHA,2
1,,NAIHATI,1
2,,PALPARA,2
3,,SIMURALI,1
4,ALLERGY,CHAKDAHA,2
...,...,...,...
110,THYROID DISORDER,PALPARA,23
111,THYROID DISORDER,SIMURALI,8
112,ULCER,CHAKDAHA,6
113,ULCER,PALPARA,4


In [389]:
localities = persons_studied["ADDRESS"].unique()

In [390]:
localities

array(['CHAKDAHA', 'KALYANI', 'NAIHATI', 'PALPARA', 'SIMURALI'],
      dtype=object)

In [391]:
diseases=df['DISEASE'].unique()

In [392]:
diseases

array(['LIVER DISEASE', 'BELLY UPSET', 'OSTEOPORESIS', 'DYSENTRY',
       'ATRIAL FIBRATION', 'HIGH CHOLESTEROL', 'ULCER', 'DIABETES',
       'HYPERTENSION', 'ARTHIRITIS', 'PCOS', 'SPINAL ISSUE', 'STROKE',
       'ASTHMA', 'LOW BP', 'ALLERGY', 'COUGH & COLD', 'THYROID DISORDER',
       'COPD', 'OVARIAN CYST', 'HAD APPENDIX OPERATION', 'CATARACT',
       'HEART DISEASE', 'CHRONIC DYSENTERY', 'BLOOD DISORDER',
       'FREQUENT FEVER', 'GALL BLADDER SURGERY', 'HAIRFALL',
       'VISION DISORDER', 'DISABILITY', 'CONSTIPATION', 'NIL', 'PAIN',
       'KIDNEY DISEASE', 'EAR INFECTION', 'BRONCHITIS', 'GLUCOMA',
       'NERVOUS ISSUE', '', 'LUNG DISEASE', 'HIGH URIC ACID',
       'BREAST CANCER', 'CALCIUM DEFICIENCY', 'IRON DEFICIENCY',
       'DEFICIENCY OF SODIUM AND POTASSIUM', 'ONE HAND PARALYSED',
       'BLOOD CANCER', 'NERVE PROBLEM', 'SPINAL CORD INJURY',
       'NIGHT BLINDNESS'], dtype=object)

In [393]:
full_index = pd.MultiIndex.from_product([diseases, localities], names=["DISEASE", "ADDRESS"])
full_df = pd.DataFrame(index=full_index).reset_index()

In [394]:
full_df

,DISEASE,ADDRESS
0,LIVER DISEASE,CHAKDAHA
1,LIVER DISEASE,KALYANI
2,LIVER DISEASE,NAIHATI
3,LIVER DISEASE,PALPARA
4,LIVER DISEASE,SIMURALI
...,...,...
245,NIGHT BLINDNESS,CHAKDAHA
246,NIGHT BLINDNESS,KALYANI
247,NIGHT BLINDNESS,NAIHATI
248,NIGHT BLINDNESS,PALPARA


In [395]:
# Merge and fill
merged = full_df.merge(affected, on=["DISEASE", "ADDRESS"], how="outer")


In [396]:
merged.head()

,DISEASE,ADDRESS,Affected_Persons
0,,CHAKDAHA,2.0
1,,KALYANI,NaN
2,,NAIHATI,1.0
3,,PALPARA,2.0
4,,SIMURALI,1.0


In [397]:
merged = merged.fillna(0)


In [398]:
merged = merged.merge(persons_studied, on="ADDRESS", how="left")
merged["Affected_Persons"] = merged["Affected_Persons"].fillna(0)
merged["Persons_Studied"] = merged["Persons_Studied"].fillna(0)

In [399]:
merged

,DISEASE,ADDRESS,Affected_Persons,Persons_Studied
0,,CHAKDAHA,2.0,151
1,,KALYANI,0.0,21
2,,NAIHATI,1.0,18
3,,PALPARA,2.0,150
4,,SIMURALI,1.0,41
...,...,...,...,...
245,VISION DISORDER,CHAKDAHA,2.0,151
246,VISION DISORDER,KALYANI,0.0,21
247,VISION DISORDER,NAIHATI,0.0,18
248,VISION DISORDER,PALPARA,0.0,150


In [400]:
# Stats
merged["Proportion"] = merged["Affected_Persons"] / merged["Persons_Studied"]
merged["% Affected"] = merged["Proportion"] * 100
merged["SD"] = ((merged["Proportion"] * (1 - merged["Proportion"])) * merged["Persons_Studied"]) ** 0.5
merged["SE"] = merged["SD"] / merged["Persons_Studied"] ** 0.5


In [401]:
merged.tail(1)

,DISEASE,ADDRESS,Affected_Persons,Persons_Studied,Proportion,% Affected,SD,SE
249,VISION DISORDER,SIMURALI,0.0,41,0.0,0.0,0.0,0.0


In [402]:
import numpy as np
summary = merged.groupby('DISEASE')['% Affected'].agg(
    Average_Percent='mean',
    SE=lambda x: x.std() / np.sqrt(len(x))
).reset_index()

In [403]:
summary.head()

,DISEASE,Average_Percent,SE
0,,2.130483,0.939423
1,ALLERGY,4.486039,3.072932
2,ARTHIRITIS,3.016522,1.004484
3,ASTHMA,2.395384,0.919748
4,ATRIAL FIBRATION,0.132450,0.132450


In [404]:
summary['T-bar'] = summary['Average_Percent'] / summary['SE']
# # Merge T-bar into final table
# merged = pd.merge(merged, summary[['DISEASE', 'T-bar']], on='DISEASE')

In [405]:
summary.head(50)

,DISEASE,Average_Percent,SE,T-bar
0,,2.130483,0.939423,2.267863
1,ALLERGY,4.486039,3.072932,1.459856
2,ARTHIRITIS,3.016522,1.004484,3.003056
3,ASTHMA,2.395384,0.919748,2.604391
4,ATRIAL FIBRATION,0.132450,0.132450,1.000000
5,BELLY UPSET,0.265784,0.162760,1.632978
6,BLOOD CANCER,0.487805,0.487805,1.000000
7,BLOOD DISORDER,0.265784,0.162760,1.632978
8,BREAST CANCER,0.133333,0.133333,1.000000
9,BRONCHITIS,2.037212,1.876038,1.085912


In [406]:
merged_duplicate = merged

In [407]:
merged_duplicate.head()

,DISEASE,ADDRESS,Affected_Persons,Persons_Studied,Proportion,% Affected,SD,SE
0,,CHAKDAHA,2.0,151,0.013245,1.324503,1.404817,0.114322
1,,KALYANI,0.0,21,0.000000,0.000000,0.000000,0.000000
2,,NAIHATI,1.0,18,0.055556,5.555556,0.971825,0.229061
3,,PALPARA,2.0,150,0.013333,1.333333,1.404754,0.114698
4,,SIMURALI,1.0,41,0.024390,2.439024,0.987730,0.154257


In [408]:
summary[['DISEASE', 'T-bar']]

,DISEASE,T-bar
0,,2.267863
1,ALLERGY,1.459856
2,ARTHIRITIS,3.003056
3,ASTHMA,2.604391
4,ATRIAL FIBRATION,1.000000
5,BELLY UPSET,1.632978
6,BLOOD CANCER,1.000000
7,BLOOD DISORDER,1.632978
8,BREAST CANCER,1.000000
9,BRONCHITIS,1.085912


In [409]:
# Merge T-bar into final table
merged_duplicate = pd.merge(merged_duplicate, summary[['DISEASE', 'T-bar']], on='DISEASE')

In [410]:
merged_duplicate.tail(30)

,DISEASE,ADDRESS,Affected_Persons,Persons_Studied,Proportion,% Affected,SD,SE,T-bar
220,SPINAL CORD INJURY,CHAKDAHA,0.0,151,0.000000,0.000000,0.000000,0.000000,1.000000
221,SPINAL CORD INJURY,KALYANI,0.0,21,0.000000,0.000000,0.000000,0.000000,1.000000
222,SPINAL CORD INJURY,NAIHATI,0.0,18,0.000000,0.000000,0.000000,0.000000,1.000000
223,SPINAL CORD INJURY,PALPARA,0.0,150,0.000000,0.000000,0.000000,0.000000,1.000000
224,SPINAL CORD INJURY,SIMURALI,1.0,41,0.024390,2.439024,0.987730,0.154257,1.000000
225,SPINAL ISSUE,CHAKDAHA,3.0,151,0.019868,1.986755,1.714759,0.139545,1.632978
226,SPINAL ISSUE,KALYANI,0.0,21,0.000000,0.000000,0.000000,0.000000,1.632978
227,SPINAL ISSUE,NAIHATI,0.0,18,0.000000,0.000000,0.000000,0.000000,1.632978
228,SPINAL ISSUE,PALPARA,3.0,150,0.020000,2.000000,1.714643,0.140000,1.632978
229,SPINAL ISSUE,SIMURALI,0.0,41,0.000000,0.000000,0.000000,0.000000,1.632978


In [412]:
merged_duplicate.to_excel("T-Score_final_4.xlsx", index=False)